In [1]:
import torch as tr
import numpy as np
import su2_chain as s
import integrators as integ

In [2]:
a = tr.normal(0.0,2.0,[4])
a = 2*a/tr.norm(a)
print(a)
U = s.Mat(a) #tr.sum(a[:,None,None] * basis,dim=0)
print(U)
print("Should be identity")
print(s.TdxT(U,U))

tensor([-0.9841, -1.4684, -0.3596, -0.8637])
tensor([[-0.4920-0.4318j,  0.1798-0.7342j],
        [-0.1798-0.7342j, -0.4920+0.4318j]])
Should be identity
tensor([[1.0000-5.9428e-09j, 0.0000-2.9802e-08j],
        [0.0000-2.9802e-08j, 1.0000+5.9428e-09j]])


In [3]:
b = tr.normal(0.0,2.0,[4])
b = 2*b/tr.norm(b)
print(b)
V = s.Mat(b) #tr.sum(a[:,None,None] * basis,dim=0)
print(V)
print("Should be identity")
print(s.TdxT(V,V))

tensor([ 0.0308, -1.9323, -0.5043, -0.1052])
tensor([[ 0.0154-0.0526j,  0.2521-0.9661j],
        [-0.2521-0.9661j,  0.0154+0.0526j]])
Should be identity
tensor([[1.-2.4296e-09j, 0.+4.6566e-10j],
        [0.+4.6566e-10j, 1.+2.4296e-09j]])


In [4]:
tr.trace(U@tr.conj(V.T)).real.item()/2-(a@b/4).item()

-5.960464477539063e-08

In [5]:
su2f = s.field([128,128],Nbatch=10)

In [6]:
su2c = s.SU2chain(beta=3.0,field_type=su2f)

In [7]:
P = su2c.refreshP()

In [8]:
U =su2c.f.hot()
U.shape

torch.Size([10, 128, 128, 2, 2])

In [9]:
a= su2c.f.coeffs(U)

In [10]:
a.shape

torch.Size([10, 128, 128, 4])

In [11]:
s.Mat(a[0,0,0,:])- U[0,0,0,:,:]

tensor([[0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]])

In [12]:
FF = su2c.f.mat(a)

In [13]:
(FF-U).norm()

tensor(0.)

In [14]:
P.shape

torch.Size([10, 128, 128, 2, 2])

In [15]:
P[...,1,0].real.mean(), P[...,1,0].real.std(),P[...,1,0].imag.mean(), P[...,1,0].imag.std()

(tensor(-6.7169e-05), tensor(0.5002), tensor(0.0005), tensor(0.5015))

In [16]:
P[...,0,0].real.mean(), P[...,0,0].real.std(),P[...,0,0].imag.mean(), P[...,0,0].imag.std()

(tensor(0.), tensor(0.), tensor(-0.0022), tensor(0.4989))

In [17]:
tr.sqrt(tr.tensor(2))

tensor(1.4142)

In [18]:
pp = 2.0*tr.einsum('kij,...ji->...k',s.tau,P)

In [19]:
pp.real.std(dim=(0,1,2))

tensor([1.0029, 1.0004, 0.9978])

In [20]:
pp.shape

torch.Size([10, 128, 128, 3])

In [21]:
ff = -tr.einsum('...k,kij->...ij',pp,s.tau)

In [22]:
ff[0,0,0]-P[0,0,0]

tensor([[0.-2.9802e-08j, 0.+0.0000e+00j],
        [0.+0.0000e+00j, -0.-2.9802e-08j]])

In [23]:
tr.trace(P[0,0,0])

tensor(0.+5.9605e-08j)

In [24]:
tr.einsum('...ij,...ij->',P,P.conj()).real

tensor(245942.3438)

In [25]:
tr.einsum('...k,...k->',pp,pp)/2

tensor(245942.3594+0.j)

In [26]:
(tr.norm(P,dim=(su2c.f.ci[0],su2c.f.ci[1])) - tr.norm(pp,dim=(len(pp.shape)-1))/tr.sqrt(tr.tensor(2))).norm()

tensor(2.7808e-05)

In [27]:
E = su2c.f.expo(P)

In [28]:
E[0,0,0] @ E[0,0,0].conj().transpose(0,1)

tensor([[1.0000e+00+0.0000e+00j, 1.0811e-08-2.9802e-08j],
        [1.0811e-08+2.9802e-08j, 1.0000e+00+0.0000e+00j]])

In [29]:
tr.eye(2,2) + P[0,0,0]@(tr.eye(2,2) + 0.5*P[0,0,0]@(tr.eye(2,2)+ 1/3*P[0,0,0]@(tr.eye(2,2) + 1/4*P[0,0,0]))), E[0,0,0]

(tensor([[ 0.6926-0.2853j,  0.6598+0.0096j],
         [-0.6598+0.0096j,  0.6926+0.2853j]]),
 tensor([[ 0.6922-0.2864j,  0.6623+0.0096j],
         [-0.6623+0.0096j,  0.6922+0.2864j]]))

In [30]:
def expo0(P):
        nn = tr.norm(P,dim=(-1,-2))/np.sqrt(2)
        E = tr.einsum('...,ij->...ij',tr.cos(nn),tr.eye(2,2)) + tr.einsum('...,...ij->...ij',tr.sin(nn)/nn,P) 
        return E

def expo(P):
        nn = tr.norm(P,dim=(-1,-2))/np.sqrt(2)
        cos=tr.cos(nn).unsqueeze(-1).unsqueeze(-1)
        sin=tr.sin(nn).unsqueeze(-1).unsqueeze(-1)
        mask = nn > 1e-8
        eye = tr.eye(2, dtype=P.dtype, device=P.device).expand_as(P)
        unit = tr.zeros_like(P)
        unit[mask] = P[mask]/nn[mask].unsqueeze(-1).unsqueeze(-1)

        return cos*eye + sin*unit

def expo2(P):
        nn = tr.norm(P,dim=(-1,-2))/np.sqrt(2)
        cos=tr.cos(nn).unsqueeze(-1).unsqueeze(-1)
        sin=tr.where(nn > 1e-8,tr.sin(nn)/nn,tr.ones_like(nn)).unsqueeze(-1).unsqueeze(-1)       
        return cos*tr.eye(2, dtype=P.dtype, device=P.device).expand_as(P) + sin*P

def expo3(P):
        nn = tr.norm(P,dim=(-1,-2))/np.sqrt(2)
        cos=tr.cos(nn)
        sin=tr.where(nn > 1e-8,tr.sin(nn)/nn,tr.ones_like(nn))       

        return tr.einsum('...,ij->...ij',cos,tr.eye(2)) + tr.einsum('...,...ij->...ij',sin,P) 

In [31]:
%timeit E = su2c.f.expo(P)
%timeit EE = expo0(P)
%timeit EEE = expo3(P)
E   = su2c.f.expo(P)
EE  = expo0(P)
EEE = expo3(P)
tr.norm(EE-E),tr.norm(EEE-E)

20.3 ms ± 278 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
21.2 ms ± 748 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.5 ms ± 242 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


(tensor(0.), tensor(0.))

In [32]:
P = su2c.refreshP()
Ken = su2c.kinetic(P)
Ven = su2c.action(U)
Hi = Ken + Ven
print("The total initial kinetic energy is: ",Ken)
print("The total initial potential energy is: ",Ven)
print("The total initial energy is: ",Hi)

The total initial kinetic energy is:  tensor([24597.0352, 24636.8438, 24500.8516, 24676.9043, 24415.0645, 24504.3477,
        24642.3398, 24556.2207, 24559.3789, 24641.1211])
The total initial potential energy is:  tensor([98188.1484, 98104.3594, 98365.6562, 98224.4375, 98282.4609, 98480.6484,
        98723.8125, 98765.5156, 98394.2969, 98328.4062])
The total initial energy is:  tensor([122785.1875, 122741.2031, 122866.5078, 122901.3438, 122697.5234,
        122985.0000, 123366.1562, 123321.7344, 122953.6719, 122969.5312])


In [33]:
dt = 0.1
k = 10
l = integ.leapfrog(su2c.force,su2f.evolveQ,k,1.0)
m = integ.minnorm2(su2c.force,su2f.evolveQ,k,1.0)
PP ,QQ  = l.integrate(P,U)
PP2,QQ2 = m.integrate(P,U)

In [34]:
KenF = su2c.kinetic(PP)
VenF = su2c.action(QQ)
Hf = KenF + VenF
print("The total initial kinetic energy is: ",KenF)
print("The total initial potential energy is: ",VenF)
print("The total initial energy is: ",Hf)
print("The total energy diff is: ",Hf-Hi)

The total initial kinetic energy is:  tensor([58480.0938, 58965.4102, 57826.8125, 58277.0898, 58200.5664, 58881.5938,
        58697.0664, 59296.6758, 58238.5273, 58292.2422])
The total initial potential energy is:  tensor([64135.1484, 63601.3672, 64875.4062, 64455.5859, 64328.6953, 63929.9414,
        64499.1719, 63850.2422, 64545.4297, 64506.8750])
The total initial energy is:  tensor([122615.2422, 122566.7812, 122702.2188, 122732.6719, 122529.2656,
        122811.5312, 123196.2344, 123146.9219, 122783.9531, 122799.1172])
The total energy diff is:  tensor([-169.9453, -174.4219, -164.2891, -168.6719, -168.2578, -173.4688,
        -169.9219, -174.8125, -169.7188, -170.4141])


In [35]:
KenF = su2c.kinetic(PP2)
VenF = su2c.action(QQ2)
Hf = KenF + VenF
print("The total initial kinetic energy is: ",KenF)
print("The total initial potential energy is: ",VenF)
print("The total initial energy is: ",Hf)
print("The total energy diff is: ",Hf-Hi)

The total initial kinetic energy is:  tensor([58721.1914, 59209.3047, 58062.1836, 58515.5273, 58439.4219, 59124.8359,
        58937.8281, 59540.6289, 58477.9570, 58533.1484])
The total initial potential energy is:  tensor([64051.8984, 63519.6484, 64792.6758, 64374.0000, 64246.2578, 63848.1094,
        64416.3750, 63768.8203, 64463.9414, 64424.4688])
The total initial energy is:  tensor([122773.0938, 122728.9531, 122854.8594, 122889.5312, 122685.6797,
        122972.9453, 123354.2031, 123309.4531, 122941.8984, 122957.6172])
The total energy diff is:  tensor([-12.0938, -12.2500, -11.6484, -11.8125, -11.8438, -12.0547, -11.9531,
        -12.2812, -11.7734, -11.9141])


In [36]:
foo = tr.einsum('kij,klm->ijlm',s.tau,s.tau)

In [37]:
foo[0,0,1,1]

tensor(0.2500+0.j)

In [38]:
2.0*tr.trace(s.tau[0]@s.tau[0].T.conj())

tensor(1.+0.j)

In [39]:
Pa=tr.tensor([2.0*tr.trace(P[0,0,0]@s.tau[0].T.conj()),2.0*tr.trace(P[0,0,0]@s.tau[1].T.conj()),2.0*tr.trace(P[0,0,0]@s.tau[2].T.conj())])

In [40]:
tr.trace(P[0,0,0]@P[0,0,0].T.conj())

tensor(1.4127+0.j)

In [41]:
(Pa*Pa).sum()/2

tensor(1.4127+0.j)

In [42]:
P[0,0,0]

tensor([[ 0.0000-0.3738j,  0.4773+0.5821j],
        [-0.4773+0.5821j,  0.0000+0.3738j]])

In [43]:
tr.norm(tr.einsum('aij,a->ij',s.tau,Pa) - P[0,0,0])

tensor(2.9802e-08)

In [44]:
su2c.kinetic(P)

tensor([24597.0352, 24636.8438, 24500.8516, 24676.9043, 24415.0645, 24504.3477,
        24642.3398, 24556.2207, 24559.3789, 24641.1211])

In [45]:
# the kinnetic energy derivative
eps = 10.0
D = tr.zeros_like(P)

for x in range(P.shape[1]):
    for y in range(P.shape[2]):
        for a in range(3):
            PPp = P.clone()
            PPm = P.clone()
            #PPp[:,x,y] = tr.einsum('ik,bkj->bij',Rp[a],PPp[:,x,y])
            #PPm[:,x,y] = tr.einsum('ik,bkj->bij',Rm[a],PPm[:,x,y])
            PPp[:,x,y] +=  eps*s.tau[a]
            PPm[:,x,y] -=  eps*s.tau[a]
            foo = (su2c.kinetic(PPp) - su2c.kinetic(PPm))/2.0/eps
            for b in range(D.shape[0]):
                D[b,x,y] += s.tau[a]*foo[b]
            
            

In [46]:
tr.norm(D-P)/tr.norm(P)

tensor(5.1800e-05)

In [47]:
# the potential energy derivative 
eps = 0.01
Rp = tr.zeros([3,2,2],dtype=tr.complex64)
Rm = tr.zeros([3,2,2],dtype=tr.complex64)
Dact = tr.zeros_like(P)
for k in range(3):
    Rp[k] = su2c.f.expo(+eps*s.tau[k].unsqueeze(0).unsqueeze(0).unsqueeze(0)).squeeze()
    Rm[k] = su2c.f.expo(-eps*s.tau[k].unsqueeze(0).unsqueeze(0).unsqueeze(0)).squeeze()
    

for x in range(P.shape[1]):
    for y in range(P.shape[2]):
        for a in range(3):
            Up = U.clone()
            Um = U.clone()
            Up[:,x,y] = tr.einsum('ik,bkj->bij',Rp[a],Up[:,x,y])
            Um[:,x,y] = tr.einsum('ik,bkj->bij',Rm[a],Um[:,x,y])
            foo = (su2c.action(Up) - su2c.action(Um))/2.0/eps
            for b in range(D.shape[0]):
                Dact[b,x,y] += s.tau[a]*foo[b]



In [48]:
Dact[0,0,0]

tensor([[ 0.0000+0.5859j,  1.1719+0.5859j],
        [-1.1719+0.5859j,  0.0000-0.5859j]])

In [49]:
F=su2c.force(U)

In [50]:
tr.norm(F+Dact)/tr.norm(F)

tensor(0.2219)